<h1 align=center><font size = 6>Coursera Data Science Capstone Project (Course 9)</font></h1> 

## W3- Segmenting and Clustering Neighbourhoods in Toronto

In this lab, the aim is to:
1. Scrape Wikipedia webpage and create a panda dataframe of Toronto neighborhoods using the postal codes.
2. Add longitude and latitudes to the neighbourhoods in the dataframe.
3. Explore and Analyze the Toronto neigbourhoods (similar to New York lab). 
3. Cluster Toronto neighborhoods (similar to New York lab).



In [1]:
# Import libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

#! conda update -n base -c defaults conda
!pip install beautifulsoup4 
!conda install lxml -y
!conda install html5lib

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0



geopy-2.0.0          | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: d

## Part 1 - Create Dataframe of Toronto neighbourhoods

Use the Wikipedia page with the Toronto neighborhood data to scrape and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format.

In [2]:
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(URL)
df=dfs[0]
print('The size of the dataframe is:',df.shape)
df.head()

The size of the dataframe is: (180, 3)


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [3]:
#Clean datafram - remove rows with "Not assigned" Boroughs

rows2drop_id=df[df['Borough']=="Not assigned"].index
df.drop(rows2drop_id,inplace=True)
df.reset_index(drop=True,inplace=True)
df.head(12)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [4]:
print('The shape of the cleaned dataframe is:',df.shape)

The shape of the cleaned dataframe is: (103, 3)


## Part 2 - Explore the Toronto neighbourhoods

(A) Add longitude and latitude to the dataframe.  


In [5]:
# Download csv file from url
!wget -q -O 'TO_Long_Lat.csv' https://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [6]:
# convert csv to dataframe
df_LongLat = pd.read_csv('TO_Long_Lat.csv')
df_LongLat.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [7]:
# Merge borough dataframe with lonituted dataframe based on postal code into a third new dataframe
df_TO = pd.merge(left=df, right=df_LongLat, left_on='Postal Code', right_on='Postal Code')
df_TO.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
